In [14]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error,r2_score
from sklearn.impute import KNNImputer

In [7]:
path_train=r'C:\Users\mohaa\Desktop\shahrzad\train.csv'
path_test=r'C:\Users\mohaa\Desktop\shahrzad\test.csv'
df=pd.read_csv(path_train,index_col='Id')
df_t=pd.read_csv(path_test,index_col='Id')
df_t.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [8]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearB

In [17]:
missing_percentage = df.isna().mean() * 100
for index, i in missing_percentage.items():
    if i >= 15:
        print(f"{index} :  {i}")

LotFrontage :  17.73972602739726
Alley :  93.76712328767123
FireplaceQu :  47.26027397260274
PoolQC :  99.52054794520548
Fence :  80.75342465753424
MiscFeature :  96.30136986301369


Case 1: You need to explain about One-hot encoding (to specify median or mean, it's better to see if we have outliers in corresponding column)

In [30]:
def pre_process(df):
    df = df.drop(['MiscFeature', 'PoolQC', 'Fence', 'Alley', 'FireplaceQu'], axis=1)
    
    # Fill numeric missing values
    num_df = df.select_dtypes(include='number')
    num_cols = num_df.columns
    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())
    
    # Fill string missing values
    str_df = df.select_dtypes(include='object')
    str_cols = str_df.columns
    for col in str_cols:
        df[col] = df[col].fillna(df[col].value_counts().idxmax())
    
    # One-hot encoding
    df = pd.get_dummies(df, columns=str_cols, drop_first=True)
    
    return df

ptr_df=pre_process(df)
ptr_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,0,1,0,0,0,1,0
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,0,1,0,0,0,1,0
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,0,1,0,0,0,1,0
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,0,1,0,0,0,0,0
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,0,1,0,0,0,1,0


Case 2: You need to explain Encoding (encoding vs one-hot encoding)to specify median or mean, it's better to see if we have outliers in corresponding column

In [19]:
def pre_process(df):
    df = df.drop(['MiscFeature', 'PoolQC', 'Fence', 'Alley', 'FireplaceQu'], axis=1)
    
    # Fill numeric missing values
    num_df = df.select_dtypes(include='number')
    num_cols = num_df.columns
    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())
    
    # Fill string missing values
    str_df = df.select_dtypes(include='object')
    str_cols = str_df.columns
    for col in str_cols:
        df[col] = df[col].fillna(df[col].value_counts().idxmax())
    
    # Encoding string columns
    for col in str_cols:
        df[col] = pd.factorize(df[col])[0]
    
    return df

ptrV2_df=pre_process(df)
ptrV2_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,0,65.0,8450,0,0,0,0,0,0,...,0,0,0,0,0,2,2008,0,0,208500
2,20,0,80.0,9600,0,0,0,0,1,0,...,0,0,0,0,0,5,2007,0,0,181500
3,60,0,68.0,11250,0,1,0,0,0,0,...,0,0,0,0,0,9,2008,0,0,223500
4,70,0,60.0,9550,0,1,0,0,2,0,...,272,0,0,0,0,2,2006,0,1,140000
5,60,0,84.0,14260,0,1,0,0,1,0,...,0,0,0,0,0,12,2008,0,0,250000
